## 1. 생성자

In [1]:
import numpy as np
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Conv2D, Conv2DTranspose, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop

In [2]:
latent_dim = 32
height = 32
width = 32
channels = 3
generator_input = Input(shape=(latent_dim,))
# 입력을 16 × 16 크기의 128개 채널을 가진 특성 맵으로 변환
x = Dense(128 * 16 * 16)(generator_input)
x = LeakyReLU()(x)
x = Reshape((16, 16, 128))(x)
# 합성곱 층을 추가
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)
# 32 × 32 크기로 업샘플링
x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = LeakyReLU()(x)
# 합성곱 층을 더 추가
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)
# 32 × 32 크기의 3개 채널을 가진 특성 맵을 생성
x = Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = Model(generator_input, x)
generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

## 2. 판별자

In [3]:
discriminator_input = Input(shape=(height, width, channels))
x = Conv2D(128, 3)(discriminator_input)
x = LeakyReLU()(x)
x = Conv2D(128, 4, strides=2)(x)
x = LeakyReLU()(x)
x = Conv2D(128, 4, strides=2)(x)
x = LeakyReLU()(x)
x = Conv2D(128, 4, strides=2)(x)
x = LeakyReLU()(x)
x = Flatten()(x)

# 드롭아웃 층을 넣는 것이 아주 중요합니다!
x = Dropout(0.4)(x)

# 분류 층
x = Dense(1, activation='sigmoid')(x)

discriminator = Model(discriminator_input, x)
discriminator.summary()

discriminator_optimizer = RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 6, 6, 128)         0   

## 3. 적대적네트워크

In [4]:
# 판별자의 가중치가 훈련되지 않도록 설정
discriminator.trainable = False

gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)

gan_optimizer = RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')
gan.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
model (Model)                (None, 32, 32, 3)         6264579   
_________________________________________________________________
model_1 (Model)              (None, 1)                 790913    
Total params: 7,055,492
Trainable params: 6,264,579
Non-trainable params: 790,913
_________________________________________________________________


## 4. Training

In [5]:
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.datasets import cifar10

# CIFAR10 데이터를 로드
(x_train, y_train), (_, _) = cifar10.load_data()

# 자동차 이미지를 선택합니다(클래스 1)
x_train = x_train[y_train.flatten() == 1]

# 데이터를 정규화합니다
x_train = x_train.reshape(
    (x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
try:
    os.mkdir('gan_cars')
except:
    print('')

save_dir = 'gan_cars/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

# 훈련 반복 시작
start = 0
for step in range(iterations):
    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 가짜 이미지를 디코딩합니다
    generated_images = generator.predict(random_latent_vectors)

    # 진짜 이미지와 연결합니다
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    # 진짜와 가짜 이미지를 구분하여 레이블을 합칩니다
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    # 레이블에 랜덤 노이즈를 추가합니다. 아주 중요합니다!
    labels += 0.05 * np.random.random(labels.shape)

    # discriminator를 훈련합니다
    d_loss = discriminator.train_on_batch(combined_images, labels)

    # 잠재 공간에서 무작위로 포인트를 샘플링합니다
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # 모두 “진짜 이미지"라고 레이블을 만듭니다
    misleading_targets = np.zeros((batch_size, 1))

    # generator를 훈련합니다(gan 모델에서 discriminator의 가중치는 동결됩니다)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(x_train) - batch_size:
      start = 0

    # 중간 중간 저장하고 그래프를 그립니다
    if step % 100 == 0:
        # 모델 가중치를 저장합니다
        gan.save_weights('gan.h5')

        # 측정 지표를 출력합니다
        print('스텝 %s에서 판별자 손실: %s' % (step, d_loss))
        print('스텝 %s에서 적대적 손실: %s' % (step, a_loss))

        # 생성된 이미지 하나를 저장합니다
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_car' + str(step) + '.png'))


스텝 0에서 판별자 손실: 0.7061386108398438
스텝 0에서 적대적 손실: 0.6738420128822327
스텝 100에서 판별자 손실: 0.5893954038619995
스텝 100에서 적대적 손실: 3.290945053100586
스텝 200에서 판별자 손실: 0.7339250445365906
스텝 200에서 적대적 손실: 0.8166136741638184
스텝 300에서 판별자 손실: 0.7115854620933533
스텝 300에서 적대적 손실: 0.7393046021461487
스텝 400에서 판별자 손실: 0.7331420183181763
스텝 400에서 적대적 손실: 0.7275669574737549
스텝 500에서 판별자 손실: 0.6993645429611206
스텝 500에서 적대적 손실: 0.7677363157272339
스텝 600에서 판별자 손실: 0.6834217309951782
스텝 600에서 적대적 손실: 0.7404698729515076
스텝 700에서 판별자 손실: 0.7145865559577942
스텝 700에서 적대적 손실: 0.7523738145828247
스텝 800에서 판별자 손실: 0.6965606808662415
스텝 800에서 적대적 손실: 0.7620114684104919
스텝 900에서 판별자 손실: 0.7383178472518921
스텝 900에서 적대적 손실: 0.7488983273506165
스텝 1000에서 판별자 손실: 0.694585919380188
스텝 1000에서 적대적 손실: 0.968920111656189
스텝 1100에서 판별자 손실: 0.7080823183059692
스텝 1100에서 적대적 손실: 0.7392157316207886
스텝 1200에서 판별자 손실: 0.6933003664016724
스텝 1200에서 적대적 손실: 0.7861016392707825
스텝 1300에서 판별자 손실: 0.6919986605644226
스텝 1300에서 적대적 손실: 0.7766755